# Exercise 5: Social Network Mining (4 Points)

## Task 5-1: Data Preparation (2 points)

Notes: 
* Since the stated goal is _"to analyze which dancers dance together in choreographies"_, but not in _which_ choreographies, we disregard the choreography type information (which could be used e. g. as edge types).
* We assume "case matrix" to be synonym with adjacency matrix (where dancers are nodes and the number of shared dances defines the strength of the edge connecting them in the graph).
* We ignore loops, i. e. edges from a vertex to itself.

In [1]:
import pandas as pd
import itertools


dancers: dict = {
    1: "Simon", 
    2: "Maria",
    3: "Frank",
    4: "Sally",
    5: "Bert",
    6: "Anna"
}
dancers_inverse = {v: k for k, v in dancers.items()}

dances: dict = {
    1: "Phantom", 
    2: "Strike1",
    3: "Solo",
    4: "Duo",
    5: "Trio"
}

df: pd.DataFrame = pd.DataFrame(
    [
        (dancers[participation[0]], dances[participation[1]])
        for participation in 
        [
            (1, 1), (2, 3), (2, 4), (2, 1), (3, 4), (3, 1), (4, 1), (4, 2), (4, 5), (5, 1), (5, 2), (5, 5), (6, 2), (6, 5), (1, 2)
        ]
    ], 
    columns=["Dancer", "Dance"]
)

dancer_pairings: list = []
    
dancer_pairings = pd.DataFrame(
    [
        (*pair, dance_id) 
        for dance_id in df.Dance.unique() 
        for pair in itertools.product(df[df.Dance == dance_id].Dancer.values, repeat=2)
        if pair[0] != pair[1]
    ],
    columns=["Dancer1", "Dancer2", "Dance"]
)

# Case matrix.
case_matrix = pd.crosstab(dancer_pairings.Dancer1, dancer_pairings.Dancer2)
print("Case matrix:")
display(case_matrix)

# .net file.
net_text = f"""
*Network
*Vertices {len(dancers)}
"""
for i in dancers:
    net_text += str(i) + " " + dancers[i] + "\n"
net_text += "*Arcs\n"

dancer_pairings_redux = dancer_pairings[["Dancer1", "Dancer2"]]
dancer_pairings_redux["cnt"] = 1
dancer_pairings_redux.Dancer1 = dancer_pairings.Dancer1.apply(lambda x: dancers_inverse[x])
dancer_pairings_redux.Dancer2 = dancer_pairings.Dancer2.apply(lambda x: dancers_inverse[x])
edge_vals = dancer_pairings_redux.groupby(["Dancer1", "Dancer2"]).sum().reset_index().values
for val in edge_vals:
    net_text += f"{val[0]} {val[1]} {val[2]}\n"

net_text += "*Edges\n"
print(".net format:")
print(net_text)

Case matrix:


Dancer2,Anna,Bert,Frank,Maria,Sally,Simon
Dancer1,,,,,,
Anna,0,2,0,0,2,1
Bert,2,0,1,1,3,2
Frank,0,1,0,2,1,1
Maria,0,1,2,0,1,1
Sally,2,3,1,1,0,2
Simon,1,2,1,1,2,0


.net format:

*Network
*Vertices 6
1 Simon
2 Maria
3 Frank
4 Sally
5 Bert
6 Anna
*Arcs
1 2 1
1 3 1
1 4 2
1 5 2
1 6 1
2 1 1
2 3 2
2 4 1
2 5 1
3 1 1
3 2 2
3 4 1
3 5 1
4 1 2
4 2 1
4 3 1
4 5 3
4 6 2
5 1 2
5 2 1
5 3 1
5 4 3
5 6 2
6 1 1
6 4 2
6 5 2
*Edges



## Task 5-2: Analysis (2 points)

We choose _local degree centrality_ as local and _global degree centrality_ as global metric for measuring dancers' "importance".

In [2]:
from scipy.sparse.csgraph import shortest_path
import numpy as np

# Local degree centrality.
print("Local degree centrality:")
dancer_pairings = dancer_pairings.drop(columns=["Dance"], errors="ignore")
dancer_pairings["degree"] = 1
display(dancer_pairings.groupby(["Dancer1"]).sum() / (len(dancers) - 1))

# Global decree centrality.
print("Global k-path centrality:")
global_degree_centrality = shortest_path(
    # Convert edge weights to distances; leave zeros to signal non-reachability.
    np.asarray([
        [elem if elem == 0 else 1 for elem in row]
        for row in case_matrix.values
    ]), 
    directed=False
).sum(axis=0)

for i, dancer in enumerate(case_matrix.columns):
    print(dancer, global_degree_centrality[i])
    
print("\nWeighted k-path centrality:")
global_degree_centrality = shortest_path(
    # Convert edge weights to distances; leave zeros to signal non-reachability.
    np.asarray([
        [elem if elem == 0 else 1 / elem for elem in row]
        for row in case_matrix.values
    ]), 
    directed=False
).sum(axis=0)

for i, dancer in enumerate(case_matrix.columns):
    print(dancer, global_degree_centrality[i])



Local degree centrality:


,degree
Dancer1,
Anna,1.0
Bert,1.8
Frank,1.0
Maria,1.0
Sally,1.8
Simon,1.4


Global k-path centrality:
Anna 7.0
Bert 5.0
Frank 6.0
Maria 6.0
Sally 5.0
Simon 5.0

Weighted k-path centrality:
Anna 5.0
Bert 3.3333333333333335
Frank 5.0
Maria 5.0
Sally 3.333333333333333
Simon 4.0


**Interpretation:**
* With _local degree centrality_, higher values can be considered more central (since this implies that a node is connected to more other nodes). In this fashion, Bert and Sally are the most central dancers.
* With _global k-path centrality_, lower values are better (since this implies that a node is quickly reachable from other nodes). Here, weights can be considered or ignored - with the former, similar to the result from the local degree centrality analysis, Bert and Sally are the most central nodes, whereas with the latter Simon is included ex aequo.

Combining results from both (or all three variants), it seems that a ranking of dancers by their centrality in this graph would be:
1. Sally, Bert
2. Simon
3. Frank, Maria
4. Anna